In [1]:
import pandas as pd
import numpy as np
import os
from missforest import MissForest
from sklearn.model_selection import train_test_split

# Importing data and modify location in the process


In [3]:
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None) 

In [4]:
# Specify the folder path 
folder_path = folder_path = 'C:/Users/PAVILION/Documents/Estudios Python/MANUAL ML/PROYECTO HOUSING MARKET/Data cities no cleaned'

# Create a dictionary to store the data
dfs = {}

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'): 
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Extract the city name (the part before the first '_')
        city_name = file_name.split('_')[0]
        
        # Assign the city name to the 'Location' column of the DataFrame
        df['Location'] = city_name
        
        # Use the file name (without extension) as the dictionary key
        df_name = os.path.splitext(file_name)[0]
        
        # Store the DataFrame in the dictionary
        dfs[df_name] = df

In [5]:
# Combine all DataFrames into a single DataFrame
data_cities = pd.concat(dfs.values(), ignore_index=True)

In [6]:
data_cities.head()

,URL,Location,Bedrooms,Bathrooms,Area,Property Type,Stratum,Price
0,https://www.properati.com.co/detalle/14032-32-ab7f-9a19e97d9cf7-1914ebd-811a-71d1,armenia,1 habitación,2 baños,42 m²,Apartamento,5.0,$ 230.000.000
1,https://www.properati.com.co/detalle/14032-32-bc1-a747f68b0dbb-18fc233-8293-7533,armenia,3 habitaciones,2 baños,56 m²,Apartamento,4.0,$ 260.000.000
2,https://www.properati.com.co/detalle/14032-32-3cb1-5a716cf88971-2758fa55-a6d1-3600,armenia,3 habitaciones,2 baños,77 m²,Apartamento,4.0,$ 210.000.000
3,https://www.properati.com.co/detalle/14032-32-b9e5-701a8b4af260-67f2a609-a92c-33d4,armenia,3 habitaciones,2 baños,77 m²,Apartamento,4.0,$ 285.000.000
4,https://www.properati.com.co/detalle/14032-32-e7fe-d82bd074042f-89803a37-ba2e-31d3,armenia,3 habitaciones,1 baño,56 m²,Apartamento,4.0,$ 228.000.000


# Data cleaning

Data cleaning always must be performed to guarantee a good EDA and modeling.

Overal goals from this process were:

1. Verify values we are working with.
2. Guarentee data consistency.
3. Check possible corrupt values.
4. Deal with missing values.

# Str cleaning

According to the nature and description of the variables:

1. Rooms, bathrooms, area and price must be numeric.
2. Stratum, property type and Location must be categorical.

Therefore:

1. All str and special characters were cleaned in numeric columns.
2. It was verified the consistency of str characters in categorical columns (capital letters, spaces, etc).

In [10]:
# Elimination of unnecessary strings
data_cities.Bedrooms = data_cities.Bedrooms.str.replace(pat = 'habitaciones', repl = "")
data_cities.Bedrooms = data_cities.Bedrooms.str.replace(pat = 'habitación', repl = "")
data_cities.Bathrooms = data_cities.Bathrooms.str.replace(pat = 'baños', repl = '')
data_cities.Bathrooms = data_cities.Bathrooms.str.replace(pat = 'baño', repl = '')
data_cities.Area = data_cities.Area.str.replace(pat = 'm²', repl = '')
data_cities.Price = data_cities.Price.str.replace(pat = '$', repl = '')
data_cities.Price = data_cities.Price.str.replace(pat = '.', repl = '')

# NaN analysis

In the data mining process, it was noticed three types of NaN data:

1. Links are not available (captcha triggered by website, properties already sold). In this case all columns are NaN.
2. Links are available but most data is missing (at least Bedroom, Bathrooms and Area are missing).
3. Links are available but only stratum is missing.

Therefore, the information obtained from item (1) was dropped. Nevertheless, from (2) and (3) can be simulated with a strong library like MissForest.

In [13]:
# Looking for NAs
for col in data_cities.columns:
    print(f"Value counts NAs for column {col}:")
    print(data_cities[col].isna().sum(axis = 0))
    print("\n")

Value counts NAs for column URL:
0


Value counts NAs for column Location:
0


Value counts NAs for column Bedrooms:
9831


Value counts NAs for column Bathrooms:
9833


Value counts NAs for column Area:
10077


Value counts NAs for column Property Type:
331


Value counts NAs for column Stratum:
19687


Value counts NAs for column Price:
331




In [14]:
# Filtering missing data from item 1
data_cities_cleaned = data_cities[data_cities['Price'].notna()]

In [15]:
# Verification of NaN after cleaning
for col in data_cities_cleaned.columns:
    print(f"Value counts NAs for column {col}:")
    print(data_cities_cleaned[col].isna().sum(axis = 0))
    print("\n")

Value counts NAs for column URL:
0


Value counts NAs for column Location:
0


Value counts NAs for column Bedrooms:
9500


Value counts NAs for column Bathrooms:
9502


Value counts NAs for column Area:
9746


Value counts NAs for column Property Type:
0


Value counts NAs for column Stratum:
19356


Value counts NAs for column Price:
0




# Value counts  after NaN analysis

This step was executed to notice possible mixing type of data and outliers. For this reason an initial value count was performed.

In [17]:
# Iterate over each column to see which values are in each column
for col in data_cities_cleaned.columns:
    print(f"Value counts for column {col}:")
    print(data_cities_cleaned[col].value_counts())
    print("\n")

Value counts for column URL:
URL
https://www.properati.com.co/detalle/14032-32-9ccc-e94f71fc74ea-18f4bb3-b802-7f26     4
https://www.properati.com.co/detalle/14032-32-f3fa-10a049cf6799-19192b6-8a7a-78a2     4
https://www.properati.com.co/detalle/14032-32-d9e3-b0b4e0fcd5b4-c23a7d07-900f-45c9    4
https://www.properati.com.co/detalle/14032-32-f1f3-38ec25ade513-18f3ead-99f0-7b84     4
https://www.properati.com.co/detalle/14032-32-4518-1574b9ce9e58-a5544723-ab7d-3428    4
                                                                                     ..
https://www.properati.com.co/detalle/14032-32-b5cc-23bdf0fea653-1916148-ba4c-7353     1
https://www.properati.com.co/detalle/14032-32-333c-ad4977ea9024-191c056-807d-7db1     1
https://www.properati.com.co/detalle/14032-32-ca03-6558779cd3ca-e700e31d-8cf0-4951    1
https://www.properati.com.co/detalle/14032-32-5533-94480855ac10-11e29c1f-b1eb-44ea    1
https://www.properati.com.co/detalle/14032-32-ac24-53e9db830d26-191d4a2-864f-78ae     1

* URL being repeated mean duplicates were mined. 
* Bedrooms and Bathrooms clearly showed outliers because they have both unrealistic and strange number of bedrooms and bathrooms, like 64 and 67 respectly. This issue was solved in the EDA section.
* Location, Stratum, Property Type and Area were fine. Nevertheless, Stratum 7 must be handled because is a category that doesn't exist oficially. Also, Property Type had a corrupt data 'Lote Industrial'.
* A glimpse in Price showed some unrealistic values (apartments or houses of 250, 380, etc) and mixed type of data ('/mes', 'consultar').

In [19]:
pd.set_option('display.min_rows', None) 

In [20]:
data_cities_cleaned = data_cities_cleaned[data_cities_cleaned['Property Type'] != 'Lote industrial']

In [21]:
# Using pd.to_numeric to find those new NaN
data_cities_cleaned['Price']= pd.to_numeric(data_cities_cleaned['Price'], errors='coerce')

In [22]:
# Counts of NaN obtained from mixed data in price
data_cities_cleaned.Price.isna().sum(axis = 0)

47

In [23]:
# Dropping new NaN
data_cities_cleaned = data_cities_cleaned.dropna(axis = 0, subset = ['Price'])

* Now that Price is fully numeric, the whole column was divided by 1000000 for better legibility of the numbers.

In [25]:
# Expressing price in millions of Colombian Pesos
data_cities_cleaned.Price = data_cities_cleaned.Price/1000000

In [26]:
# Sorting data again by location
data_cities_cleaned_filtered  = data_cities_cleaned.sort_values('Location')

# Finding duplicates

The drop_duplicates method were executed to eliminate all the duplicates based on the URLs.

In [29]:
# Elimination of duplicates 
data_cities_cleaned_filtered.drop_duplicates(subset = ['URL'], inplace = True)

In [30]:
data_cities_cleaned_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74464 entries, 0 to 77648
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   URL            74464 non-null  object 
 1   Location       74464 non-null  object 
 2   Bedrooms       65663 non-null  object 
 3   Bathrooms      65661 non-null  object 
 4   Area           65424 non-null  object 
 5   Property Type  74464 non-null  object 
 6   Stratum        55766 non-null  float64
 7   Price          74464 non-null  float64
dtypes: float64(2), object(6)
memory usage: 5.1+ MB


# Elimination of stratum 7

As explained before, the following command filtered this corrupt data.

In [33]:
data_cities_cleaned_filtered = data_cities_cleaned_filtered[data_cities_cleaned_filtered.Stratum != 7]

# Final adjustments before filling of NaN

For better data presentation, the following steps were executed:

1. Put the first letters of the cities in capital
2. Change 'Property Type' for 'Type'
3. Translate apartamento and casa to their respective english words

In [36]:
# Capitalizing location column
data_cities_cleaned_filtered.loc[:, 'Location'] = data_cities_cleaned_filtered['Location'].str.capitalize()

In [37]:
data_cities_cleaned_filtered.head()

,URL,Location,Bedrooms,Bathrooms,Area,Property Type,Stratum,Price
0,https://www.properati.com.co/detalle/14032-32-ab7f-9a19e97d9cf7-1914ebd-811a-71d1,Armenia,1,2,42,Apartamento,5.0,230.0
2486,https://www.properati.com.co/detalle/14032-32-6719-a30f7bdb1793-18f4cfd-9d51-7cda,Armenia,3,3,90,Casa,4.0,330.0
2487,https://www.properati.com.co/detalle/14032-32-9b98-9fd465c31a8d-191ec29-94e6-7799,Armenia,4,2,180,Casa,4.0,449.0
2488,https://www.properati.com.co/detalle/14032-32-ffc-34bd36bd85da-9dc9124a-a883-4fe0,Armenia,3,3,778,Casa,6.0,1344.0
2489,https://www.properati.com.co/detalle/14032-32-d8d6-c0dc42d0ef12-472dccd1-aac1-3852,Armenia,NaN,NaN,NaN,Casa,NaN,1200.0


In [38]:
# Changing column name
data_cities_cleaned_filtered.rename(columns={'Property Type':'Type'}, inplace=True)

In [39]:
data_cities_cleaned_filtered.head()

,URL,Location,Bedrooms,Bathrooms,Area,Type,Stratum,Price
0,https://www.properati.com.co/detalle/14032-32-ab7f-9a19e97d9cf7-1914ebd-811a-71d1,Armenia,1,2,42,Apartamento,5.0,230.0
2486,https://www.properati.com.co/detalle/14032-32-6719-a30f7bdb1793-18f4cfd-9d51-7cda,Armenia,3,3,90,Casa,4.0,330.0
2487,https://www.properati.com.co/detalle/14032-32-9b98-9fd465c31a8d-191ec29-94e6-7799,Armenia,4,2,180,Casa,4.0,449.0
2488,https://www.properati.com.co/detalle/14032-32-ffc-34bd36bd85da-9dc9124a-a883-4fe0,Armenia,3,3,778,Casa,6.0,1344.0
2489,https://www.properati.com.co/detalle/14032-32-d8d6-c0dc42d0ef12-472dccd1-aac1-3852,Armenia,NaN,NaN,NaN,Casa,NaN,1200.0


In [40]:
data_cities_cleaned_filtered.replace(to_replace = {'Type' : {'Apartamento' : 'Apartment', 'Casa' : 'House'}}, inplace = True )

In [41]:
# Verification of the previous steps
data_cities_cleaned_filtered.head()

,URL,Location,Bedrooms,Bathrooms,Area,Type,Stratum,Price
0,https://www.properati.com.co/detalle/14032-32-ab7f-9a19e97d9cf7-1914ebd-811a-71d1,Armenia,1,2,42,Apartment,5.0,230.0
2486,https://www.properati.com.co/detalle/14032-32-6719-a30f7bdb1793-18f4cfd-9d51-7cda,Armenia,3,3,90,House,4.0,330.0
2487,https://www.properati.com.co/detalle/14032-32-9b98-9fd465c31a8d-191ec29-94e6-7799,Armenia,4,2,180,House,4.0,449.0
2488,https://www.properati.com.co/detalle/14032-32-ffc-34bd36bd85da-9dc9124a-a883-4fe0,Armenia,3,3,778,House,6.0,1344.0
2489,https://www.properati.com.co/detalle/14032-32-d8d6-c0dc42d0ef12-472dccd1-aac1-3852,Armenia,NaN,NaN,NaN,House,NaN,1200.0


# Filling missing values rigorously with missForest

MissForest is a project found in https://pypi.org/project/MissForest/. It is the implementation of its respective package in R and is used for providing an accurate method to imput missing-mixed type of data.

For its correct execution, it was necessary to perform some steps, keeping in mind the recomendation from the authors:

1. Verify the data type of each column.
2. Label correctly each data type of each column.

In [44]:
# Check data type of each column
data_cities_cleaned_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74443 entries, 0 to 77648
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   URL        74443 non-null  object 
 1   Location   74443 non-null  object 
 2   Bedrooms   65643 non-null  object 
 3   Bathrooms  65641 non-null  object 
 4   Area       65405 non-null  object 
 5   Type       74443 non-null  object 
 6   Stratum    55745 non-null  float64
 7   Price      74443 non-null  float64
dtypes: float64(2), object(6)
memory usage: 5.1+ MB


From  the previous step, it can be noticed that only Price has the correct type. 

In [46]:
# Transformation to categorical data
data_cities_cleaned_filtered[['Location', 'Type', 'Stratum']] = data_cities_cleaned_filtered[['Location', 'Type', 'Stratum']].astype('category')

In [47]:
# Transformation to numerical data
data_cities_cleaned_filtered[['Bedrooms', 'Bathrooms', 'Area']] = data_cities_cleaned_filtered[['Bedrooms', 'Bathrooms', 'Area']].apply(pd.to_numeric, errors='coerce')

Finally, the elaboration of the missforest code was done with the help of the train and test split from sklearn. The column URL is no longer needed, and its presence is troublesome for the execution of the script, therefore it was dropped.

In [49]:
data_cities_cleaned_filtered.drop(columns = 'URL', inplace = True)

In [50]:
# Split dataset into train and test sets
train, test = train_test_split(data_cities_cleaned_filtered, train_size=0.8, random_state=42)

# Reset index before applying miceForest
train.reset_index(drop = True, inplace = True)
test.reset_index(drop = True, inplace = True)

# MissForest code
mf = MissForest()
train_imputed = mf.fit_transform(train, categorical=['Location', 'Type', 'Stratum'])
test_imputed = mf.transform(test)

# Convert the NumPy array back to DataFrame and ensure the index is consistent
train_imputed = pd.DataFrame(train_imputed, columns=train.columns).reset_index(drop=True)
test_imputed = pd.DataFrame(test_imputed, columns=test.columns).reset_index(drop=True)

100%|██████████| 5/5 [00:08<00:00,  1.74s/it]


In [51]:
# Merging train and test data in a final dataframe
final_data = pd.concat([train_imputed, test_imputed], axis = 0)

In [52]:
# Verification of the previous steps
final_data.head()

,Location,Bedrooms,Bathrooms,Area,Type,Stratum,Price
0,Bogota,3.000000,4.000000,356.000000,House,6.0,1150.0
1,Barranquilla,3.000000,5.000000,199.000000,House,6.0,900.0
2,Pereira,1.000000,1.000000,35.000000,Apartment,4.0,195.0
3,Armenia,4.000000,3.000000,145.000000,House,3.0,300.0
4,Cali,3.152335,2.885554,99.548979,Apartment,4.0,195.0


In [53]:
# Iterate over each column to see which values are in each column
for col in final_data.columns:
    print(f"Value counts for column {col}:")
    print(final_data[col].value_counts())
    print("\n")

Value counts for column Location:
Location
Medellin         9912
Bogota           9847
Cali             8462
Pereira          7985
Barranquilla     7068
Cartagena        6092
Manizales        6030
Armenia          3686
Cucuta           3347
Santa-marta      2473
Monteria         2337
Bucaramanga      2063
Villavicencio    1468
Ibague           1338
Popayan           985
Tunja             500
Neiva             473
Sincelejo         208
Valledupar        108
Pasto              61
Name: count, dtype: int64


Value counts for column Bedrooms:
Bedrooms
3.000000     30814
4.000000     12015
2.000000      9812
5.000000      4631
1.000000      3204
6.000000      2081
7.000000       982
8.000000       811
9.000000       373
10.000000      320
3.152335       235
3.308968       163
11.000000      162
12.000000      144
3.372344       139
2.961170       109
3.287526        84
2.944660        76
3.149087        68
13.000000       67
3.057479        61
3.112558        60
3.233111        58
3.250121 

* Some values in bedrooms and bathrooms are decimals or negatives and this is not possible. Therefore the np.rint method was executed in the next command as long with a filter of number of bedrooms > 0.

In [55]:
final_data['Bedrooms'] = np.rint(final_data['Bedrooms'])
final_data['Bathrooms'] = np.rint(final_data['Bathrooms'])

In [56]:
final_data = final_data[final_data.Bedrooms > 0 ]

In [57]:
final_data.sort_values('Location', ascending = True, inplace = True)
final_data.head()

,Location,Bedrooms,Bathrooms,Area,Type,Stratum,Price
54033,Armenia,4.0,3.0,222.0,Apartment,6.0,970.0
20564,Armenia,5.0,3.0,147.0,House,3.0,350.0
6122,Armenia,3.0,3.0,115.0,House,4.0,470.0
39615,Armenia,3.0,2.0,100.0,House,3.0,190.0
5079,Armenia,3.0,2.0,68.0,Apartment,5.0,300.0


# Saving the cleaned dataframe into a new csv file

Data is now proper to perform an EDA and supervised/unsupervised learning. It was saved in a new file

In [59]:
final_data.to_csv('data_cities_complete.csv', index = False)

In [60]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74429 entries, 54033 to 14888
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Location   74429 non-null  object 
 1   Bedrooms   74429 non-null  float64
 2   Bathrooms  74429 non-null  float64
 3   Area       74429 non-null  float64
 4   Type       74429 non-null  object 
 5   Stratum    74429 non-null  float64
 6   Price      74429 non-null  float64
dtypes: float64(5), object(2)
memory usage: 4.5+ MB


In [61]:
final_data.head()

,Location,Bedrooms,Bathrooms,Area,Type,Stratum,Price
54033,Armenia,4.0,3.0,222.0,Apartment,6.0,970.0
20564,Armenia,5.0,3.0,147.0,House,3.0,350.0
6122,Armenia,3.0,3.0,115.0,House,4.0,470.0
39615,Armenia,3.0,2.0,100.0,House,3.0,190.0
5079,Armenia,3.0,2.0,68.0,Apartment,5.0,300.0
